In [10]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import nrrd
import copy

sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("Set1"))

In [19]:
patients=['FJ', 'BE', 'MJ1', 'HI']
top_dir="/Volumes/ssd750/radiomics/sbrt_radiomics/data"

for patient in patients:
    print(patient)
    patient_dir = os.path.join(top_dir,patient)
    
    for dir_item in os.listdir(patient_dir):
        d = os.path.join(patient_dir, dir_item)
        
        if os.path.isdir(d):
            radiomics_dir = os.path.join(patient_dir, d, 'radiomics')
            
            for csv_item in os.listdir(radiomics_dir):
                if os.path.splitext(csv_item)[1] == '.csv':
                    df_temp = pd.read_csv(os.path.join(radiomics_dir, csv_item))
                    df_temp['timepoint'] = str(os.path.basename(dir_item))
                    df_temp['patient'] = patient
                    
                    if 'df' in locals():
                        df = pd.concat([df, df_temp])
                    else:
                        df = copy.deepcopy(df_temp)

df.reset_index(drop=True, inplace=True)

FJ
BE
MJ1
HI


In [20]:
# Asign dummy variable for time ordering

# Assume we only have Pre and PostN where N is NOT a left-zero-padded integer
timepoints_order = list(set(df['timepoint']))
timepoints_order.remove('Pre')
timepoints_order = [int(x.replace('Post','')) for x in timepoints_order]
timepoints_order.sort()
timepoints_order = ['Post%d' % x for x in timepoints_order]
timepoints_order.insert(0,'Pre')

timepoints_map = {idx: timepoint for timepoint,idx in enumerate(timepoints_order)}
df['time_index'] = df['timepoint'].map(timepoints_map)
df.reset_index(drop=True, inplace=True)

In [21]:
# Add actual times since treatment
df_times = pd.read_csv(os.path.join(top_dir, 'times.csv'))
df_times

,patient,timepoint,date
0,BE,Pre,1/4/2012
1,BE,Post1,6/12/2012
2,BE,Post2,10/2/2012
3,BE,Post3,4/2/2013
4,BE,Post4,10/1/2013
5,BE,Post5,4/8/2014
6,HI,Pre,1/14/2015
7,HI,Post1,6/10/2015
8,HI,Post2,12/2/2015
9,HI,Post3,4/6/2016


In [24]:
fig_dir="/Users/matter/Desktop/20220217_radiomics"

fig = plt.figure()

cols_to_plot = list(df.columns[df.columns.str.startswith("original")])
for feature in cols_to_plot:
   
    sns.relplot(data=df[(df['mask']=='Aorta_wall_JM.nrrd') | (df['mask']=='Aorta_wall_JM2.nrrd')],
                    x='time_index',
                    y=feature,
                    hue='dose_bin',
                    style='patient',
                    row='mask',
                    markers=True,
                    dashes=True,
                    sort=True,
                    kind="line")

    # "reverse" the timepoint map for xtick labels
    plt.xticks(list(timepoints_map.values()), list(timepoints_map.keys()))

    plt.xlabel("Time point")
    plt.suptitle(feature)

    plt.gcf().set_size_inches(15, 8)
    
    plt.savefig(os.path.join(fig_dir, feature + '.pdf'))    
    plt.cla()
    plt.close()

<Figure size 432x288 with 0 Axes>

In [23]:
df

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,success,patient,mask,dose_bin,timepoint,time_index
0,v3.0.1,1.19.5,2.1.1,1.1.1,3.6.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ec6da6fbbaab47a46e295f54058f797687145524,3D,"(1.0, 1.0, 1.0)",...,0.000054,3256.991719,0.002696,0.685647,True,FJ,Aorta_-_Grow__1.00mm.nrrd,0.0 to 1.0 Gy,Post1,1
1,v3.0.1,1.19.5,2.1.1,1.1.1,3.6.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ec6da6fbbaab47a46e295f54058f797687145524,3D,"(1.0, 1.0, 1.0)",...,0.000106,9193.381724,0.002588,7.060133,True,FJ,Aorta_-_Grow__1.00mm.nrrd,1.0 to 30.0 Gy,Post1,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,FJ,Aorta_-_Grow__1.00mm.nrrd,30.0 to 50.0 Gy,Post1,1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,FJ,Aorta_-_Grow__1.00mm.nrrd,50.0 to 60.0 Gy,Post1,1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,FJ,Aorta_-_Grow__1.00mm.nrrd,60.0 to inf Gy,Post1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2902,v3.0.1,1.19.5,2.1.1,1.1.1,3.6.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},52853f919b891bdcb816139338a1be0ab8a22830,3D,"(1.0, 1.0, 1.0)",...,0.000459,61.065875,0.015985,0.035122,True,HI,Aorta_wall_JM.nrrd,0.0 to 1.0 Gy,Pre,0
2903,v3.0.1,1.19.5,2.1.1,1.1.1,3.6.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},52853f919b891bdcb816139338a1be0ab8a22830,3D,"(1.0, 1.0, 1.0)",...,0.000261,582.155304,0.001729,1.173645,True,HI,Aorta_wall_JM.nrrd,1.0 to 30.0 Gy,Pre,0
2904,v3.0.1,1.19.5,2.1.1,1.1.1,3.6.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},52853f919b891bdcb816139338a1be0ab8a22830,3D,"(1.0, 1.0, 1.0)",...,0.001590,511.730254,0.005691,6.812784,True,HI,Aorta_wall_JM.nrrd,30.0 to 50.0 Gy,Pre,0
2905,v3.0.1,1.19.5,2.1.1,1.1.1,3.6.6,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},52853f919b891bdcb816139338a1be0ab8a22830,3D,"(1.0, 1.0, 1.0)",...,0.001584,56.744225,0.008690,0.199260,True,HI,Aorta_wall_JM.nrrd,50.0 to 60.0 Gy,Pre,0


In [22]:
set(df[df.patient=='HI'].timepoint)

{'Post1', 'Post2', 'Post3', 'Post4', 'Post5', 'Post6', 'Post7', 'Post8', 'Pre'}